In [14]:
def calculate_standard_error(t_statistic, parameter_estimate):
    """
    Calculate the standard error given a t-statistic and the parameter estimate.
    
    Parameters:
    t_statistic (float): The t-statistic value.
    parameter_estimate (float): The estimated parameter value (e.g., mean difference).
    
    Returns:
    float: The calculated standard error.
    """
    if t_statistic == 0:
        raise ValueError("t_statistic cannot be zero.")
    return abs(parameter_estimate / t_statistic)



In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smp
from wildboottest.wildboottest import wildboottest
from data_gen_homogenous_AR1 import generate_staggered_law_ar1_data

alpha = 0.05  # Significance level
num_simulations = 1000
rho = 0.8
num_individuals = 1
N = 50
T = 20

reject_count = 0  # Counter for the number of rejections
true_beta1_value = 0   # Initialising the counters
bias_values = []
squared_error_values = []
standard_error_values =[]
beta1_estimates = []

np.random.seed(42) # setting seed for reproducibility


for _ in range(num_simulations):

    data = generate_staggered_law_ar1_data( N, T, rho, num_individuals) # this time the data generation is inside the loop to ensure everytime new data is generated

    states = data['state'].unique()
    
    treatment_states = np.random.choice(states, size=len(states)//2, replace=False)   # To give the treatment to exactly half the states in a staggered approach

    treatment_years = np.random.choice(range(5, 15), size=len(treatment_states), replace=True)  # Assigning treatment year to each treatment state, staggered between 5 and 15th time period

    state_to_treatment_year = dict(zip(treatment_states, treatment_years))

    # Adding a treatment column to the DataFrame
    
    data['TREATMENT'] = data.apply(lambda x: 1 if x['state'] in treatment_states and x['time'] >= state_to_treatment_year[x['state']] else 0, axis=1)

    model = smp.ols(formula="value ~ TREATMENT + 1 + state_2 + state_3 + state_4 + state_5 + state_6 + state_7 + state_8 + state_9 + state_10 + state_11 + state_12 + state_13 + state_14 + state_15 + state_16 + state_17 + state_18 + state_19 + state_20 + state_21 + state_22 + state_23 + state_24 + state_25 + state_26 + state_27 + state_28 + state_29 + state_30 + state_31 + state_32 + state_33 + state_34 + state_35 + state_36 + state_37 + state_38 + state_39 + state_40 + state_41 + state_42 + state_43 + state_44 + state_45 + state_46 + state_47 + state_48 + state_49 + state_50 + time_1 + time_2 + time_3 + time_4 + time_5 + time_6 + time_7 + time_8 + time_9 + time_10 + time_11 + time_12 + time_13 + time_14 + time_15 + time_16 + time_17 + time_18 + time_19", data=data)

    result = model.fit()

    wildboottest_results = wildboottest(model, param="TREATMENT", cluster=data['state'], B= 400, bootstrap_type='11', show = False)
    wildboottest_results = pd.DataFrame(wildboottest_results)
    p_value = wildboottest_results['p-value'].iloc[0]
    param = result.params['TREATMENT']
    t_stat = wildboottest_results['statistic'].iloc[0] 

    
    # Checking  if null hypothesis for beta1 is rejected

    if p_value < alpha:
        reject_count += 1

    beta1_estimates.append(param)
    
    bias = np.mean(beta1_estimates) - true_beta1_value

    squared_error = (param - true_beta1_value) ** 2
    squared_error_values.append(squared_error)

    # Extracting the standard error of the 'TREATMENT' coefficient 

    standard_error = calculate_standard_error(t_stat, param)
    standard_error_values.append(standard_error)


type1_error = reject_count/ num_simulations
bias = np.mean(bias)
mse = np.mean(squared_error_values)
rmse = np.sqrt(mse)
average_standard_error = np.mean(standard_error_values)


std_error_beta_distribution = np.std(beta1_estimates)  # Calculating the standard error of the distribution of beta

confidence_interval = (
    np.mean(beta1_estimates) - 1.96 * std_error_beta_distribution,
    np.mean(beta1_estimates) + 1.96 * std_error_beta_distribution
)


print(f"Number of times null hypothesis is rejected: {reject_count} out of {num_simulations} simulations")
print(f"Type 1 Error: {type1_error}")
print(f"Bias for Coefficient of Treatment : {bias}")
print(f"MSE for Coefficient of Treatment : {mse}")
print(f"RMSE for Coefficient of Treatment : {mse}")
print(f"Average Standard Error: {average_standard_error}")
print(f"Standard Error of the beta_1 distribution: {std_error_beta_distribution}")
print(f'The confidence interval is {confidence_interval[0] , {confidence_interval[1]}}')

sns.kdeplot(beta1_estimates, bw_adjust=0.5, fill=True)
plt.xlabel(r'$\beta_1$ Estimates')  # Using LaTeX for beta_1
plt.ylabel('Density')
plt.title(r'Distribution of $\beta_1$ Estimates')


mean_beta1_estimate = np.mean(beta1_estimates) # Calculating mean of beta1 estimates for marking on plot

# Adding vertical lines for the confidence interval and the mean beta1 estimate

plt.axvline(x=confidence_interval[0], color='red', linestyle='--', label='Lower CI')
plt.axvline(x=confidence_interval[1], color='green', linestyle='--', label='Upper CI')
plt.axvline(x=mean_beta1_estimate, color='blue', linestyle='-', label='Mean Beta1 Estimate')

plt.legend(loc='lower left', bbox_to_anchor=(1, 0))

plt.grid(True, which='both', linestyle='-', linewidth=0.5)
plt.show()

